In [ ]:
# 📘 Step 1: Install and Import Required Libraries
# Install Huggingface Transformers and datasets if not already installed
!pip install transformers

In [ ]:
# 📘 Step 2: Import Required Libraries
import random  # For generating discount codes
import pandas as pd  # For tracking order status
from transformers import pipeline  # For AI-based query response

In [ ]:
# 📘 Step 3: Load customer order data from CSV
# Directly load the file from /content/data/train.csv
csv_path = '/content/data/train.csv'
orders_df = pd.read_csv(csv_path, dtype={'ID': str})
orders_df['ID'] = orders_df['ID'].str.strip().str.lower()  # Remove whitespace and convert to lowercase
orders_df['ID'] = orders_df['ID'].str.replace('0x', '', regex=False)  # Clean up the '0x' prefix if present
# Debug: Print available order IDs for validation
print("\n📋 Available Order IDs:", orders_df['ID'].unique())


📋 Available Order IDs: ['4607' 'b379' '5d6d' ... '4f8d' '5eee' '5fb2']


In [ ]:
# 📘 Step 4: Initialize HuggingFace model for AI responses
print("\n⏳ Loading EleutherAI/gpt-neo-1.3B model. This may take a few minutes...")
assistant_model = pipeline("text-generation", model="EleutherAI/gpt-neo-1.3B", device=-1)  # -1 uses CPU




⏳ Loading EleutherAI/gpt-neo-1.3B model. This may take a few minutes...


Device set to use cpu


In [ ]:
# 📘 Step 5: Define Utility Functions
def check_order_status(order_id):
    """Check the order status from the CSV file."""
    order_id = order_id.strip().lower()  # Clean user input
    order = orders_df.loc[orders_df['ID'] == order_id]
    if not order.empty:
        return order.iloc[0]['Time_taken(min)']
    else:
        return 'Order not found'

def predict_delivery_time(order_id):
    """Predict delivery time for a specific order from the CSV data."""
    order_id = order_id.strip().lower()  # Clean user input
    order = orders_df.loc[orders_df['ID'] == order_id]
    if not order.empty:
        time_taken = int(order.iloc[0]['Time_taken(min)'].replace('(min) ', ''))
        return time_taken
    else:
        return 'Order not found'

def generate_discount_code():
    """Generate a random discount code for the customer."""
    return f'DISC-{random.randint(1000, 9999)}'

def get_weather_and_traffic_info(order_id):
    """Get weather and traffic information for a specific order."""
    order_id = order_id.strip().lower()  # Clean user input
    order = orders_df.loc[orders_df['ID'] == order_id]
    if not order.empty:
        weather_condition = order.iloc[0]['Weatherconditions'].replace('conditions ', '')
        traffic_condition = order.iloc[0]['Road_traffic_density']
        return {"weather": weather_condition, "traffic": traffic_condition}
    return {"weather": "unknown", "traffic": "unknown"}

def generate_delivery_status_message(order_id):
    """Generate a message about the delivery status of an order."""
    order_id = order_id.strip().lower()  # Clean user input
    order = orders_df.loc[orders_df['ID'] == order_id]

    if order.empty:
        return "We couldn't find your order. Please check the Order ID and try again."

    status = "Out for delivery"
    weather_and_traffic = get_weather_and_traffic_info(order_id)
    weather = weather_and_traffic['weather']
    traffic = weather_and_traffic['traffic']
    delivery_time = predict_delivery_time(order_id)
    delay = 10  # Example fixed delay for demonstration
    total_delivery_time = delivery_time + delay

    return (f"Your order will be delivered in {total_delivery_time} minutes. It is {delay} minutes later than the usual time due to "
            f"{weather} weather and {traffic} traffic in your area. We apologize for the inconvenience.")

def get_llm_response(user_input, session_context):
    """Generate an AI-powered response using EleutherAI's GPT-Neo model, considering context and CSV data."""
    try:
        # Check if the user is asking for specific data like order status, weather, or traffic
        order_id = None
        csv_data = ""

        # If order tracking is mentioned, add order details to session context
        if 'track my order' in user_input.lower():
            order_id = input("🔍 Please enter your Order ID: ").strip().lower()
            if order_id in orders_df['ID'].values:
                status_message = generate_delivery_status_message(order_id)
                csv_data = f"Order ID {order_id}: {status_message}"
            else:
                csv_data = f"Order ID {order_id} not found in the data."

        # If delivery time is requested, add delivery time data to session context
        elif 'delivery time' in user_input.lower():
            order_id = input("🔍 Please enter your Order ID: ").strip().lower()
            if order_id in orders_df['ID'].values:
                delivery_time = predict_delivery_time(order_id)
                csv_data = f"Order ID {order_id}: Estimated delivery time is {delivery_time} minutes."
            else:
                csv_data = f"Order ID {order_id} not found in the data."

        # If weather or traffic info is requested, add relevant data to session context
        elif 'weather' in user_input.lower() or 'traffic' in user_input.lower():
            order_id = input("🔍 Please enter your Order ID: ").strip().lower()
            if order_id in orders_df['ID'].values:
                weather_traffic_info = get_weather_and_traffic_info(order_id)
                csv_data = f"Order ID {order_id}: Weather is {weather_traffic_info['weather']}, Traffic is {weather_traffic_info['traffic']}."
            else:
                csv_data = f"Order ID {order_id} not found in the data."

        # If 'order' is mentioned, check if the user wants to track or place a new order
        elif 'order' in user_input.lower():
            session_context += f" User mentioned 'order'. Let's ask if they want to track or place a new order."
            csv_data = "Please specify if you want to track your order or place a new one."

        # Add CSV data to the session context
        full_input = session_context + " " + user_input + " " + csv_data

        # Generate the response using LLM
        response = assistant_model(full_input,
                                   max_new_tokens=100,  # Set the number of tokens to generate
                                   num_return_sequences=1,
                                   truncation=True,  # Ensure input is truncated to max_length
                                   pad_token_id=50256)  # Explicitly set pad_token_id for open-end generation
        generated_text = response[0]['generated_text']

        # Update session context with the latest interaction (including the AI's response)
        session_context += " " + user_input + " " + csv_data + " " + generated_text
        return generated_text, session_context

    except Exception as e:
        print(f"Error fetching LLM response: {e}")
        return "Sorry, something went wrong. Please try again later.", session_context


In [ ]:
# 📘 Step 6: Customer Interaction Logic with Context Caching
def simulate_customer_interaction():
    print("\n👋 Welcome to Food Delivery Assistant! How can I help you today? (Type your request below)")

    session_context = ""  # Initialize empty session context

    while True:
        user_input = input("🗣️ You: ")

        # Exit condition
        if 'exit' in user_input.lower() or 'quit' in user_input.lower():
            print("Goodbye! Have a great day!")
            break

        # Check if order tracking is mentioned in user input
        if 'track my order' in user_input.lower():
            order_id = input("🔍 Please enter your Order ID: ").strip().lower()
            if order_id in orders_df['ID'].values:
                status_message = generate_delivery_status_message(order_id)
                print(f"📦 {status_message}")
                session_context += f" User asked to track order {order_id}. Assistant provided: {status_message}"
            else:
                print("❌ Order ID not found. Please try again.")
                session_context += f" User asked to track order {order_id}, but it was not found."

        # Check if placing a new order is mentioned
        elif 'place a new order' in user_input.lower() or 'new order' in user_input.lower():
            order_id = input("🔍 Please enter a new Order ID: ").strip().lower()
            if order_id in orders_df['ID'].values:
                distance = float(input("🚚 Enter the delivery distance in km: "))
                estimated_time = round(distance * 10)  # Example formula: 10 minutes per km
                print(f"⏳ The estimated delivery time for your order is {estimated_time} minutes.")

                # Optionally, provide discount code for new order
                discount_code = generate_discount_code()
                print(f"🎉 Use the discount code {discount_code} for your next order!")
                session_context += f" User placed a new order {order_id}. Estimated time: {estimated_time} minutes. Discount code provided: {discount_code}"
            else:
                print("❌ Order ID not found. Please check and try again.")
                session_context += f" User tried placing a new order {order_id}, but it was not found."

        # Weather and traffic related inquiries
        elif 'weather' in user_input.lower() or 'traffic' in user_input.lower():
            order_id = input("🔍 Please enter your Order ID to check the status: ").strip().lower()
            if order_id in orders_df['ID'].values:
                weather_traffic_info = get_weather_and_traffic_info(order_id)
                print(f"🌤️ Current weather: {weather_traffic_info['weather']}")
                print(f"🚗 Traffic condition: {weather_traffic_info['traffic']}")
                session_context += f" User asked about weather and traffic for order {order_id}. Weather: {weather_traffic_info['weather']}, Traffic: {weather_traffic_info['traffic']}"
            else:
                print("❌ Order ID not found. Please check and try again.")
                session_context += f" User asked about weather and traffic for order {order_id}, but it was not found."

        # If user asks a general question, get an LLM response with data from CSV
        else:
            ai_response, session_context = get_llm_response(user_input, session_context)
            print(f"🤖 AI Assistant: {ai_response}")

simulate_customer_interaction()


👋 Welcome to Food Delivery Assistant! How can I help you today? (Type your request below)
🗣️ You: order
🤖 AI Assistant:  User mentioned 'order'. Let's ask if they want to track or place a new order. order Please specify if you want to track your order or place a new one.

A:

The order line of a line, is not the same thing as an order placed, or an order that's been approved by the customer.  I believe that order = order.  Orders don't exist until a customer places an order.  The order (order line) is not a transaction that can be placed into the system, and only has any value until it's approved.


🗣️ You: track order
🤖 AI Assistant:  User mentioned 'order'. Let's ask if they want to track or place a new order. order Please specify if you want to track your order or place a new one.  User mentioned 'order'. Let's ask if they want to track or place a new order. order Please specify if you want to track your order or place a new one.

A:

The order line of a line, is not the same thing

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Error fetching LLM response: index out of range in self
🤖 AI Assistant: Sorry, something went wrong. Please try again later.
🗣️ You: Exit
Goodbye! Have a great day!
